In [1]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
import PySAM.Pvwattsv8 as pvwatts
import numpy as np

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Fetch the weather data from OpenMeteo API
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": -33.6421493,  # Latitude for Worcester, South Africa
    "longitude": 19.4488646,  # Longitude for Worcester, South Africa
    "start_date": "2024-09-27",
    "end_date": "2024-10-11",
    "hourly": ["temperature_2m", "wind_speed_10m", "shortwave_radiation", "diffuse_radiation", "direct_normal_irradiance"]
}
responses = openmeteo.weather_api(url, params=params)

# Process the response to extract hourly data
response = responses[0]
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(1).ValuesAsNumpy()
hourly_shortwave_radiation = hourly.Variables(2).ValuesAsNumpy()
hourly_diffuse_radiation = hourly.Variables(3).ValuesAsNumpy()
hourly_direct_normal_irradiance = hourly.Variables(4).ValuesAsNumpy()

# Create a dataframe from the hourly data
hourly_data = {
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    ),
    "temperature_2m": hourly_temperature_2m,
    "wind_speed_10m": hourly_wind_speed_10m,
    "shortwave_radiation": hourly_shortwave_radiation,
    "diffuse_radiation": hourly_diffuse_radiation,
    "direct_normal_irradiance": hourly_direct_normal_irradiance
}
hourly_dataframe = pd.DataFrame(data=hourly_data)

# Clean the dataframe by filling or dropping NaN values
hourly_dataframe = hourly_dataframe.replace([np.inf, -np.inf], np.nan).dropna()

# Add a minute column with all values set to zero
hourly_dataframe['minute'] = 0

# Convert the cleaned dataframe to the format needed for PySAM
solar_resource_data = {
    'lat': -33.64,  # Latitude for Worcester, South Africa
    'lon': 19.44,   # Longitude for Worcester, South Africa
    'tz': 2,        # Timezone offset (UTC+2 for South Africa)
    'elev': 230,    # Elevation in meters (approximate for Worcester)
    'year': hourly_dataframe['date'].dt.year.tolist(),
    'month': hourly_dataframe['date'].dt.month.tolist(),
    'day': hourly_dataframe['date'].dt.day.tolist(),
    'hour': hourly_dataframe['date'].dt.hour.tolist(),
    'minute': hourly_dataframe['minute'].tolist(),
    'dn': hourly_dataframe['direct_normal_irradiance'].tolist(),  # DNI
    'df': hourly_dataframe['diffuse_radiation'].tolist(),         # DHI
    'gh': hourly_dataframe['shortwave_radiation'].tolist(),       # GHI
    'tdry': hourly_dataframe['temperature_2m'].tolist(),          # Temperature
    'wspd': hourly_dataframe['wind_speed_10m'].tolist()           # Wind Speed
}

# Setup the PVWatts model
pv_system = pvwatts.default('PVWattsSingleOwner')

# Configure the PV system
pv_system.SolarResource.solar_resource_data = solar_resource_data
pv_system.SystemDesign.system_capacity = 27  # 30 kW system
pv_system.SystemDesign.module_type = 0       # Standard module
pv_system.SystemDesign.array_type = 0        # Fixed - Open Rack
pv_system.SystemDesign.tilt = 11             # 15-degree tilt
pv_system.SystemDesign.azimuth = 329.07        # 180 (south) + 10 deg west of north

# Set system losses
pv_system.SystemDesign.losses = 14  # System losses as percentage

# Run the simulation
pv_system.execute()

# Extract the hourly AC power output in kW
hourly_ac_output = pv_system.Outputs.ac

# Create a dataframe to show the date and PV production
pv_production_df = pd.DataFrame({
    "date": hourly_dataframe["date"],
    "ac_power_kw": hourly_ac_output
})

# Display the date range of the simulation and PV production dataframe
print(f"PV production date range: {pv_production_df['date'].min()} to {pv_production_df['date'].max()}")
print(pv_production_df)


PV production date range: 2024-09-27 00:00:00+00:00 to 2024-10-11 23:00:00+00:00
                         date  ac_power_kw
0   2024-09-27 00:00:00+00:00          0.0
1   2024-09-27 01:00:00+00:00          0.0
2   2024-09-27 02:00:00+00:00          0.0
3   2024-09-27 03:00:00+00:00          0.0
4   2024-09-27 04:00:00+00:00          0.0
..                        ...          ...
355 2024-10-11 19:00:00+00:00          0.0
356 2024-10-11 20:00:00+00:00          0.0
357 2024-10-11 21:00:00+00:00          0.0
358 2024-10-11 22:00:00+00:00          0.0
359 2024-10-11 23:00:00+00:00          0.0

[360 rows x 2 columns]


In [3]:
import plotly.express as px

# Create the plot using Plotly
fig = px.line(
    pv_production_df, 
    x='date', 
    y='ac_power_kw', 
    title='Hourly PV Production for the Hollywood Plant',
    labels={'date': 'Date', 'ac_power_kw': 'AC Power Output (kW)'}
)

# Update layout for better readability
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='AC Power Output (kW)',
    template='plotly_white',
    hovermode='x'
)

# Show the plot
fig.show()


In [4]:
import plotly.express as px

# Resample the DataFrame to get daily sums
daily_pv_production = pv_production_df.resample('D', on='date').sum().reset_index()

# Create the plot using Plotly
fig = px.bar(
    daily_pv_production, 
    x='date', 
    y='ac_power_kw', 
    title='Daily Sum of PV Production for the Hollywood Plant',
    labels={'date': 'Date', 'ac_power_kw': 'Daily AC Power Output (kWh)'}
)

# Update layout for better readability
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Daily AC Power Output (kWh)',
    template='plotly_white',
    hovermode='x'
)

# Show the plot
fig.show()


In [2]:
import PySAM.ResourceTools as rt

print(dir(rt))


['FetchResourceFiles', 'HTTPAdapter', 'Retry', 'SAM_CSV_to_solar_data', 'SRW_to_wind_data', 'URDBv7_to_ElectricityRates', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'certifi', 'cf', 'copy', 'csv', 'defaultdict', 'io', 'json', 'np', 'os', 'pd', 'requests', 'warnings']


In [2]:
pip install retry_requests


  Obtaining dependency information for retry_requests from https://files.pythonhosted.org/packages/b1/f3/8ce908497bebbc2790ef06240a2c0fb28c096abb59062d88f85090464a5f/retry_requests-2.0.0-py3-none-any.whl.metadata
  Using cached retry_requests-2.0.0-py3-none-any.whl.metadata (2.6 kB)
Using cached retry_requests-2.0.0-py3-none-any.whl (15 kB)
Note: you may need to restart the kernel to use updated packages.
